- [Sidebar: let’s talk about `asyncio`](#sidebar-lets-talk-about-asyncio)
- [Introducing OpenAI Agents SDK](#introducing-openai-agents-sdk)
- [1_lab1 : Our first look at OpenAI Agents SDK](./1_lab1.ipynb)
  - [Link 1_Lab1](./1_lab1.ipynb)
- [vibe coding](#-vive-coding)
- [Automated sales outreach](#automated-sales-outreach)
- [2_Lab2 : Our first Agentic Framework project](#2_lab2)
  - [Link 2_Lab2](../../notebooks/week2_openai/community_contributions/2_lab2_with_resend_email.ipynb)

---


## Sidebar: let’s talk about `asyncio`

Everything that you’ve built up over the course of **/week2\_openai** is all about the [OpenAI Agents SDK](https://openai.github.io/openai-agents-python/), formerly known as `SWARM`. But before we get into the SDK, it’s important to talk about asynchronous Python: `asyncio`.

* All the Agent Frameworks use asynchronous Python.
* You can get by ignoring it, but it will always bother you slightly.
* Bite the bullet! Spend 30 minutes on the guide—you will thank me!

Asynchronous programming is common across all agentic frameworks. You *can* survive by memorizing a couple of rules and applying them blindly, but that’s unsatisfactory. If you spend just half an hour understanding what’s really happening, you’ll thank yourself later. You’ll run into asynchronous Python again and again, and being comfortable with it will make a big difference. There are guides you can follow, and I’ll give you a high-level summary here.

### The short version

- **All your methods and functions start `async`** The most important rule: anytime you write a function that will be used asynchronously, start it with `async def`.

- **Anytime you call them, use `await`** When you call an async function, you must use the `await` keyword. This is the basic rule, and if you just follow this, you can get most code to work, even if you don’t understand all the internals.

- **AsyncIO provides a lightweight alternative to threading or multiprocessing** AsyncIO is a way of writing Python code that achieves concurrency, similar in effect to multithreading, but much lighter. In classic multithreading, you rely on the operating system to switch between threads, each running at the same time. That comes with a lot of complexity and “baggage.” AsyncIO, introduced in Python 3.5, achieves concurrency entirely at the code level—without real OS threads or multiprocessing.

Because AsyncIO is so lightweight, you can run thousands or even tens of thousands of tasks at once, using very few resources. This is especially valuable when your code spends a lot of time waiting on input/output (I/O), like network calls.

**Why is this important for agent frameworks and LLMs?**

If you’re working with large language models (LLMs), especially via paid APIs like OpenAI, most of your time is spent waiting for network responses. With async code, other parts of your program can continue running while one part is blocked, waiting for a model to respond. In multi-agent systems, where many agents may be hitting different APIs at once, using async is essential for performance and scalability.

All of the agent frameworks we’ll look at use AsyncIO for this reason.

### Under the hood: coroutines, `async def`, and `await`

**Functions defined with `async def` are called coroutines**
If you use `async def` to define a function, you’re actually creating a coroutine—a special kind of function that Python can pause and resume. Most people still call them functions, but technically, they’re coroutines.

**Calling a coroutine doesn’t execute it immediately—it returns a coroutine object**
When you call an async function, you don’t get the result right away. Instead, you get a coroutine object. No code runs yet!

**To actually run a coroutine, you must `await` it, which schedules it for execution within an event loop**
To get the result from a coroutine, you must use `await` in front of it. This tells Python to run the coroutine in the event loop and return the result when it’s finished.

**While a coroutine is waiting (e.g., for I/O), the event loop can run other coroutines**
The event loop can only execute one coroutine at a time. But if a coroutine gets stuck waiting for I/O (like waiting for a response from OpenAI), the event loop pauses that coroutine and runs another one that’s ready to go. This allows thousands of tasks to run efficiently, as long as much of the work is I/O-bound.

### Example: The short version

---
```python
async def do_some_processing() -> str:
    # Do some work
    return "done!"

result = await do_some_processing()
```

* Here, the function starts with `async def`.
* When calling it, you must use `await`.
* This pattern covers 95% of real async Python usage in agentic frameworks.


### Example: Understanding coroutines and awaiting

---
```python
async def do_some_processing() -> str:
    return "done!"

# Calling the function returns a coroutine object (nothing happens yet)
my_coroutine = do_some_processing()

# Awaiting the coroutine actually runs it and gives the result
my_result = await my_coroutine
```


You might think that just calling `do_some_processing()` would run the function, but it doesn’t. It only returns a coroutine object. To actually run it, you must await it.

You can simplify even further by writing:


```python
my_result = await do_some_processing()
```


This is what you’ll do in most real code.

### Richer example: running multiple coroutines concurrently

---
```python
results = await asyncio.gather(
    do_some_processing(),
    do_other_processing(),
    do_yet_more_processing()
)
```


Here, `asyncio.gather()` is used to schedule multiple coroutines at once. The event loop will start all three, and whenever one is blocked waiting on I/O, another can run. The results come back as a list.

This is Python’s way of implementing “fake” multithreading at the code level. It’s not real OS-level threading, but it’s often just what you need for highly concurrent, I/O-bound programs like LLM agent frameworks.

---

**Key points to remember**

* Use `async def` for every function that may need to run concurrently.
* Use `await` whenever you call one of these async functions.
* Just calling the function does not run it; it only creates a coroutine object.
* Use constructs like `asyncio.gather` to run multiple coroutines at once.

If you understand these rules and can recognize the code patterns in the examples, you’ll be productive with async Python in LLM/agent projects.

> Spend a bit of time with the guide and try the examples—this is the most valuable investment you’ll make for working with modern agentic frameworks in Python!

---



## Introducing OpenAI Agents SDK

The OpenAI Agents SDK (formerly known as `SWARM`) is the framework you’ll use to build intelligent LLM agents in a highly flexible and efficient way. Before we dive into technical details, here’s why it stands out:

* **Lightweight and flexible**
  The SDK is extremely lightweight and remarkably flexible. Unlike more “opinionated” frameworks that force you to work in a certain way, OpenAI Agents SDK gives you maximum freedom to structure your agents and workflows as you prefer.

* **Stays out of the way**
  This framework doesn’t get in your way or clutter your code. All the repetitive boilerplate and JSON-handling you’d otherwise have to do is completely abstracted for you.

* **Makes common activities easy**
  Everyday tasks—like using tools, coordinating agent interactions, or applying controls—become trivial. The SDK automates all those low-level mechanics that otherwise slow you down and distract from the core logic.


### Minimal Terminology

OpenAI Agents SDK uses a deliberately minimal, clear vocabulary. There are just three main concepts you need to know:

* **Agents** represent LLMs encapsulated with a specific role or function in your solution.
* **Handoffs** are the interactions between agents. Whenever an agent “hands off” work or information to another, that’s a handoff.
* **Guardrails** are the controls and restrictions you put in place to make sure the agent behaves as expected and doesn’t go off track. (This term is also common in general software engineering.)

### Three Steps

Running an agent with OpenAI Agents SDK is as simple as following these three core steps:

1. **Create an instance of Agents**
   Instantiate your agent, configuring its purpose and core settings.

2. **Use `with trace()` to track the agents**
   Use the `with trace()` context to log all agent interactions. This makes monitoring and debugging straightforward, and integrates seamlessly with OpenAI’s monitoring tools. While optional, it’s the recommended approach.

3. **Call `runner.run()` to run the agents**
   Use `runner.run()` to actually execute the agent. Note: this is an async function (a coroutine), so you’ll need to `await` it to trigger execution and collect the result.

That’s really all there is to it—no complicated jargon, no convoluted rules. The core concepts are minimal and direct, and the SDK makes it easy to focus on agent logic, not plumbing.


OpenAI Agents SDK is my favorite framework because it strikes the perfect balance between flexibility, simplicity, and power. While other frameworks have their strengths and may be best for certain advanced cases, in most projects you’ll both start and finish with OpenAI Agents SDK. And, as you move on to more advanced projects (such as MCP in week six), you’ll return to this SDK to leverage its full capabilities.

Remember: all agent frameworks come with their own terminology and core ideas, but OpenAI Agents SDK stands out for its clarity, minimalism, and its ability to abstract away repetitive busywork—so you can focus on building smarter agents.

---

**In summary:**

* Create your agent.
* Trace and monitor interactions.
* Run with `runner.run()` and use `await`.

The SDK handles everything else for you. Now, let’s walk through these steps in practice.



--- 


Here is your full text **restructured and organized** — without losing a single word — and formatted as a clean, readable **Jupyter Markdown section** (no icons, no extra decorations, just clarity and order):

---

## 🎥 vive coding

**My tips for successful vibing**

* Good vibes – prompt well – ask short answer and latest APIs for today’s date.
* Vibe but verify – ask 2 LLMs the same question.
* Step up the vibe – ask to break down your request into independently testable steps.
* Vibe and validate – ask an LLM then get another LLM to check.
* Vibe with variety – ask for 3 solutions to the same problem, pick the best.

**Sidebar: What is Vive Coding?**

So, that concludes our very first foray into OpenAI Agents SDK, but before we wrap up Week 2 Day 1, I did want to have a second sidebar with you, and this time it's on the entertaining topic of **vive coding**, which is a term coined by the legendary Andrej Karpathy, who described this in, I think it was an ex-post that went viral, about the way that he was enjoying coding with LLMs and getting so much done in a way that you would sort of let the LLM generate some code and sort of go with it, tweak it a bit, generate some more, and just make so much progress in this sort of mode of working, this ad hoc vive coding way of navigating around things like new frameworks.



**Why Vive Coding Works**

I think this is wonderful, and I strongly encourage vive coding, and I imagine that most of you are very good at it. I did want to give a few tips that I think are important to do it well, because I think it's easy to do vive coding and to get led astray by LLMs and get yourself in trouble and get stuck, which is very unpleasant. So, I have five tips to leave you with, but before we get into more detail with OpenAI Agents SDK, and here they are:

**1. Good Vibes**

It's important to spend time getting your prompt to the LLM to be really good, that you can reuse lots of times.
You should ask for short answers. LLMs tend to be quite verbose in the way that they're in their code that they generate.
They like packaging everything with lots of exception handlers, and they tend to do things in quite a long-winded way.
Try and ask it to come up with concise, clean code, and also mention today's date and say, please make sure that you use APIs that are current as of this date.
Otherwise, LLMs have a nasty tendency to use older APIs because that was in a lot of their training data. So, explicitly prompt for that.

**2. Vibe but Verify**

Don’t just ask an LLM a question and go with it. Ask a couple of LLMs.
So, I often ask the same question to ChatGPT and to Claude, and I have them both up.
I ask the question because I'll learn from both of the answers.
Often, one of them is too long-winded or is missing the point of it, and one of them will be spot on.
And so, asking a couple or maybe even three so that you're verifying what answers you're getting is a really good technique.

**3. Step Up the Vibe**

This is saying, and I think this is such a great one.
Sometimes students send me problems saying, “I'm stuck with this,” and they send me 200 lines of code and say, “It’s not working.”
It’s immediately obvious that this is vive coding — you can tell it was LLM-generated and it’s unwieldy, often full of bugs.
And I come back and say: It’s no good generating 200 lines of code and then saying, “It’s broken.”
Instead:

* Always try to get LLMs to generate function-by-function.
* Break it into small, independently testable parts — like 10 lines at a time.
* Think of dividing your problem into bite-sized chunks.

> **Tip**: If you don't know how to break it down, ask an LLM:
>
> "I’m trying to solve the following problem \[...] — please list 4–5 small steps where each can be tested independently."
>
>Then ask for code (and tests) for each step one by one.
> Build your full solution 10 working lines at a time.

**4. Vibe and Validate**

Similar to “vibe but verify”, but here the goal is **refinement**.

* Ask an LLM a question and get an answer.
* Then ask another LLM (or even the same one):

  > “Is this a good answer? Can it be improved? More concise? Clearer? Are there any bugs?”

It often improves the result.
This mirrors a common **agentic design pattern**: evaluator + optimizer.
You can do it manually with two LLMs, inspired by agentic ideas.

**5. Vibe with Variety**

Don’t just ask: “Can you generate code for this?”
Instead, say:

> “Give me 3 different solutions to the same problem.”

Why?

* Forces the LLM to think differently.
* Encourages multiple perspectives on the same task.
* Often gives better solutions.

Also ask for explanations of the differences and rationale.
That way:

* You’ll understand better.
* The LLM is forced to reason through its decisions.
* You stay connected to what’s going on.

**Final Note**

This leads to one final, implicit rule:

> Always ask the LLM to explain things clearly if you don’t understand.

**Vive coding is super fun, productive, and powerful.**
But if you don’t follow what’s happening, it becomes painful and frustrating when something breaks.
**Stay in touch with the logic. Understand every step.**

---


## Automated sales outreach

Here is your full text **organized and clarified** into clean Jupyter-style Markdown (no extra icons or emojis, and no words lost). This format keeps everything structured and readable — ideal for study, documentation, or teaching.

---

## Three Layers of Agentic Architecture

We're going to be building **three different pieces**, or layers, of **agentic architecture**:

1. **Basic Agent Workflow**

   * Start simple.
   * A straightforward sequence of agent calls.

2. **Agent with Tool Use**

   * Add complexity.
   * Introduce an agent that can **use a tool**.
   * Recall: we did this manually in Week 1 using JSON and boilerplate.

3. **Agents Calling Other Agents**

   * The most advanced.
   * Two ways to implement this:

     * **Agents as Tools**.
     * **Handoffs** — the special construct introduced earlier.

* Keep this **three-layer structure** in mind.
* This is how we’ll be building up complexity in our lab.
* A lot of coding ahead — let’s get started.

---

## 2_Lab2

**What We Will Do** 

- [Link 2_Lab2](2_lab2.ipynb)

* **Agent Workflow**
  A sequence of agents performing actions.

* **Use Tools to Call Functions**
  Introduce tools that agents can invoke — just like functions.

* **Collaboration with Tools and Handoffs**
  Combine agents and tools.
  Use **handoffs** to let agents interact or delegate to one another.



